In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import re
import copy
import pandas as pd

kiwi = Kiwi(typos='basic')

In [ ]:
query = '목사' # 분석 대상 키워드
df = pd.read_csv(f'API블로그{query}.csv')  # 그 키워드에 해당하는 csv 불러오기 (블로그)
#df = pd.read_csv(f'Selenium_news{query}.csv') (뉴스)
df = df.dropna(subset=['content'],ignore_index=True) # 본문(content) 스크래핑 결과가 Nan인 값을 제거

In [ ]:
# 각 종교별 키워드를 리스트에 담아서 불용어 처리에 사용 (스크래핑 검색 단어가 많이 추출되는 거는 당연해서 분석의미가 없음)
# 블로그 글 특성상 자주 나오지만 분석에 도움되지 않는 단어도 리스트로 담아 저장 (모델 수행 결과를 바탕으로 경험적으로 추가)
gidok_stop = ['기독교','교회','하나님','개신교','목사','예수']
chunju_stop = ['천주교','수녀님','성당','하느님','신부님','교황']
bul_stop = ['불교','부처님','절','스님','승려','보살']
blog_stop = ['이웃','본문','추가','복사','기타','분','곳','번']

In [ ]:
stopwords = Stopwords()
stops = gidok_stop + blog_stop # 분석 대상의 불용어와 블로그 공통 불용어를 불용어리스트에 추가
for i in stops:
    stopwords.add((i,'NNG'))
    stopwords.add((i,'NNP'))

In [ ]:
def preprocess_korean(text, analyzer=kiwi, stopwords=stopwords):
    my_text = str(copy.copy(text))
    my_text = my_text.replace('\n', ' ') # 줄바꿈 문자 제거
    my_text = kiwi.space(my_text) # 띄어쓰기 교정
    sents = kiwi.split_into_sents(my_text) # 문장 토큰화
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    all_result = []
    for sent in sents:
        token_result = kiwi.tokenize(sent.text, stopwords=stopwords) # 단어 토큰화 및 불용어 제거
        tmp = []
        for i in token_result:
            if i.tag in ['NNG','NNP']: # 보통명사와 고유명사만 추출
                tmp.append(i)
        token_result = kiwi.join(tmp) # 다시 문장으로 결합
        token_result = p.sub(' ', token_result) # 정규표현식 사용해 한글만 남김
        all_result.append(token_result) # 전체 문장들을 리스트에 저장
    
    all_result = ' '.join(all_result) # 전체 문장을 문자열에 연결해 본문 내의 모든 단어들이 하나의 문자열이 되게 설정

    return all_result

In [ ]:
df['pre_content'] = df['content'].apply(lambda x: preprocess_korean(x)) # 전처리기 적용

In [ ]:
df.to_csv(f'pre_{query}.csv', index=False)  # 전처리된 결과를 담은 csv 저장